# Real time area light

## Light transport equation (LTE in abbrev.)

Firstly, let's look at the LTE
$$L_o(W_o, P) = L_e(W_o, P) + \int{L_i(W_i, P)}{f_r(W_o, W_i)}\overline{\langle W_o, W_i \rangle} d{W_i}$$
It's a simplified version of radiative transfer equation which involve integral differential equation. Let's make our life easier and focus on LTE.

As we can see in following picture, $L_o$ and $L_i$ term is interchangeable in following relationship.
![li_lo](screenshots/Li_Lo.png)  

$$L_i(W_i, P) = L_o(-W_i, T(P, W_i)), where \space T(P, \vec{W)} = P + t * \vec{W}$$

Plugin this equation into LTE and simplify the notation, we have
$$L(W, P) = L_e(W, P) + \int{L(-W_i, T(P, W_i))}{f_r(W, W_i)}\overline{\langle W, W_i \rangle} d{W_i}$$

## Analytic solution for diffusion area light on lambert shading model

There are several types of area light that used widely in computer graphics.
* Sphere light
* Disk light
* Rectangle light
* Polygonal light

Let's look into sphere light for detail analysis. We will start with 2D space for simplicity and get familiar with Jupyter which will use quite ofen in our daily work.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

r = 2
P = np.array([2.4, 2.6])
d = np.linalg.norm(P)

fig, ax = plt.subplots()

def plot_sphere(center, radius):
    theta = np.arange(0, np.pi*2, 0.01)
    cos_theta = np.cos(theta) * radius
    sin_theta = np.sin(theta) * radius
    ax.plot(center[0] + cos_theta, center[1] + sin_theta, color='k')

def plot_line(p0, p1):
    ax.plot([p0[0], p1[0]], [p0[1], p1[1]], color='k')

plt.xlim(-5, 5)
plt.ylim(-0, 5)
ax.axhline(y=0, color='k')
ax.axvline(x=0, color='k')
ax.set_aspect('equal')
ax.grid(True, which='both')
plot_sphere(P, r)
plot_sphere([0,0], 1)

theta_0 = np.arctan2(P[1], P[0])
delta_theta = np.arcsin(r / d)
min_theta = theta_0 - delta_theta
max_theta = theta_0 + delta_theta
analytic_result = -np.cos(max_theta) + np.cos(min_theta)
print("Analytic result = ", analytic_result)

plot_line([0,0], [np.cos(theta_0)*d, np.sin(theta_0)*d])
plot_line([0,0], [np.cos(max_theta)*d, np.sin(max_theta)*d])
plot_line([0,0], [np.cos(min_theta)*d, np.sin(min_theta)*d])

mc_result = 0
dTheta = 0.001
for theta in np.arange(0, np.pi, dTheta):
    cos_theta = np.sin(theta)
    if theta >= min_theta and theta <= max_theta:
        mc_result += cos_theta
mc_result *= dTheta
print("Monte carlo result = ", mc_result)

plt.show()

It's a showcase that demonstrate what we can do with data visualization and numeric computing.  
From now on, We will start our study in 3D space! Inigo Quilez made a very detailed analysis about integrate cosine function over sphere projection in 3D in this [blog post](http://www.iquilezles.org/www/articles/sphereao/sphereao.htm). However it doesn't account for horizon clipping, the correct solution can be found in [radiation view factors](http://webserver.dmt.upm.es/~isidoro/tc3/Radiation%20View%20factors.pdf), "Patch to a sphere: Tilted" configuration

Other very important light source are polygonal light. Here are a screenshots from Unity Adam demo.  
![screenshot_ltc_1](screenshots/ltc_1.png)  

As we can see the result are very impressive, the polygonal light is working with material variation for both specular and diffuse term. It's surprising that the polygonal light has analytic solution with lambert diffuse model which solved by Johann Heinrich Lambert in 18th century. You can take a look at [Deriving the analytical formula for a diffuseresponse from a polygonal light source](http://miciwan.com/misc/diffuse_area_der.pdf) and [Geometric Derivation of the Irradiance of Polygonal Lights](https://t.co/uZxgHJ3Upz?amp=1) for step by step derivation.

$$E(p_1, ..., p_n) = \frac 1 {2 \pi} \sum_{i=1}^{n} acos(\langle p_i, p_j \rangle) \langle {\frac {p_i \times p_j} {\| {p_i \times p_j} \|} }, {\begin{bmatrix} 0\\ 0\\ 1\\ \end{bmatrix}} \rangle \quad \textrm{where} \quad j = i + 1$$

## Linearly transform cosine


Before we take closer look at LTC, let's write down our notation first.  
$D_o$: original function to be transformed, clamped cosine in our case  
$W_o$: direction before transformatoin, on hemisphere domain  
$P_o$: polygonal before transformation  
$D$: LTC transformed function, an approximation to our microfacet BRDF  
$W$: direction after transformatoin, on hemisphere domain  
$P$:polygonal after transformation  
$M$: matrix that transform $W_o$ to $W$  
$\Omega$: hemisphere domain  

The idea of LTC is very simple, let's say we have $D_o(W_o)$ that can be any spherical function, for every point on spherical, apply a 3x3 matrix $M$, will create a new spherical function $D(W)$

In [ ]:
import numpy as np
import ipyvolume as ipv
from matplotlib import cm, colors
import ipywidgets as widgets

import os
import sys
sys.path.insert(0, os.path.abspath('../pycode'))
import spherical as sph
import LTC
import importlib
importlib.reload(sph)
importlib.reload(LTC)

In [ ]:
# clamped cosine
def D_orig(Wi):
    return np.maximum(Wi[2], 0)

fig = sph.spherical_plot3d(D_orig, num_samples=256)
ipv.xyzlim(-1, 1)
ipv.show()


sph.spherical_integral(D_orig, num_samples=1024, hemisphere=False) / np.pi

In [ ]:
import numpy as np
import ipyvolume as ipv
from matplotlib import cm, colors
import ipywidgets as widgets

fig = ipv.figure(lighting=False)

def plot_ltc_func(a, b, c, d):
    theta, phi = sph.meshgrid_spherical_coord(64)
    Wi = sph.spherical_dir(theta, phi)
    
    ltc = LTC.LTC(a, b, c, d, 1)
    vals = ltc.eval(Wi)

    # normalize the value for better visualization
    vals /= np.max(vals)
    ipv.plot_mesh(Wi[0], Wi[2], Wi[1], wireframe=False, color=cm.coolwarm(vals))
#     ipv.plot_mesh(x*vals, z*vals, y*vals, wireframe=False, color=cm.coolwarm(vals))

    # plot lines in order to visualize the geometry transform
    sample_points = np.array([[0, 0, 2], [-1, -1, 2], [-1, 1, 2], [1, 1, 2], [1, -1, 2]])
    for p in sample_points:
        p = np.dot(ltc.M, p)
        p /= np.linalg.norm(p)
        p *= 2
        ipv.plot([0, p[0]], [0, p[2]], [0, p[1]])


def plot(a, b, c, d):
    fig.meshes.clear()
    fig.scatters.clear()
    plot_ltc_func(a, b, c, d)
    ipv.xyzlim(-2, 2)


aSlider = widgets.FloatSlider(min=0.01, max=1, step=0.01, value=1)
bSlider = widgets.FloatSlider(min=-4, max=4, step=0.01, value=0)
cSlider = widgets.FloatSlider(min=0.01, max=1, step=0.01, value=1)
dSlider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0)
widgets.interact(plot, a=aSlider, b=bSlider, c=cSlider, d=dSlider)
ipv.show()

With those graph and code, we should have basic understanding about LTC. Let's formalize the equation and explore some proerty of LTC.

### Linear transformation
Apply a transformation by a square matrix is a linear transform if the matrix determinant is not 0, which means the shape of a line/triangle is the same after transformation.

### Closed form expression
By apply change of variable, we have a closed form expression.  
$$D(W) = D_o(W_o) \frac{\delta W_o}{\delta W}$$
$$\frac{\delta W_o}{\delta W} = \frac{|M^{-1}|}{{|{M^{-1}}{W}|}^3}$$

### Integration
The integration of LTC is equal to original function after transformation.
$$ \int_{\Omega} D(W) d{W} =\int_{\Omega} D_o(W_o) d{W_o}$$
We can verify this by integrate $D_{ltc}$, the result should be 1.   
If we integrate over polygonal, we get following equation
$$ \int_{P} D(W) d{W} =\int_{P_o} D_o(W_o) d{W_o}$$

Here is a screenshot from original paper:  
![integrate_over_polygonal](screenshots/integrate_over_polygonal.png)

As we are interested in use LTC to solve our light transport equation, let's visualize our BRDF function.

In [ ]:
import numpy as np
import ipyvolume as ipv
from matplotlib import cm, colors
import ipywidgets as widgets

fig = ipv.figure(lighting=False)


def D_GGX(Wh, alpha):
    cosTheta = Wh[2]
    cosTheta2 = cosTheta * cosTheta
    a2 = alpha * alpha
    d = cosTheta2 * (a2 - 1) + 1
    return a2 / (np.pi * d * d)

def SmithG(V, a):
    a2 = a * a
    cosTheta = V[2]
    cosTheta2 = cosTheta * cosTheta
    return 2 * cosTheta / (cosTheta + np.sqrt(a2 + (1 - a2) * cosTheta2))

def Gvis_GGX(Wo, Wi, alpha):
    NdotV = Wo[2]
    NdotL = Wi[2]
    a2 = alpha*alpha;
    G_V = NdotV + np.sqrt( (NdotV - NdotV * a2) * NdotV + a2 );
    G_L = NdotL + np.sqrt( (NdotL - NdotL * a2) * NdotL + a2 );
    return 1 / ( G_V * G_L );

def normalizeVec3(v):
    l = np.sqrt(v[0]**2 + v[1]**2 + v[2]**2)
    return np.array([v[0]/l, v[1]/l, v[2]/l])

def dotVec3(v0, v1):
    return v0[0]*v1[0] + v0[1]*v1[1] + v0[2]*v1[2]

def halfVec(Wo, Wi):
    Wh = Wi
    Wh[0] += Wo[0]
    Wh[1] += Wo[1]
    Wh[2] += Wo[2]
    Wh = normalizeVec3(Wh)
    return Wh

def BRDF(Wo, Wi, alpha):
    Wh = halfVec(Wo, Wi)
    G = Gvis_GGX(Wo, Wi, alpha)
    D = D_GGX(Wh, alpha)
    return D * G * np.maximum(Wi[2], 0)


def plot(thetaO, roughness):
    alpha = roughness * roughness
    
    fig.meshes.clear()
    fig.scatters.clear()

    theta, phi = meshgrid_spherical_coord(256)
    x, y, z = spherical_dir(theta, phi)

    Wo = np.array([0, np.sin(thetaO), np.cos(thetaO)])
    Wi = np.array([x, y, z])

    vals = BRDF(Wo, Wi, alpha)

    # normalize the value for better visualization
    vals /= np.max(vals)
    ipv.plot_mesh(x, z, y, wireframe=False, color=cm.coolwarm(vals))
#     ipv.plot_mesh(x*vals, z*vals, y*vals, wireframe=False, color=cm.coolwarm(vals))
    ipv.xyzlim(-2, 2)


thetaSlider = widgets.FloatSlider(min=0.01, max=np.pi/2, step=0.01, value=0)
roughnessSlider = widgets.FloatSlider(min=0.01, max=1, step=0.01, value=0.3)
widgets.interact(plot, thetaO=thetaSlider, roughness=roughnessSlider)
ipv.show()

Base on our observation, it seems possible to find coefficients that make LTC an good approximation of our BRDF. The problem now become find a set of coefficients (a,b,c,d) to minimize following error measurement.
$$error(a, b, c, d) = \int_\Omega ({D\_ltc(W, a, b, c, d)} - {BRDF(W)})^2 d(W) $$

In [ ]:
import numpy as np
from scipy import optimize


def reflectVec3(Wo, Wh):
    proj = dotVec3(Wo, Wh)
    Wi = Wh * (2 * proj)
    Wi[0] -= Wo[0]
    Wi[1] -= Wo[1]
    Wi[2] -= Wo[2]
    return normalizeVec3(Wi)

thetaO = np.pi * (0.0/180)
Wo = np.array([np.sin(thetaO), 0, np.cos(thetaO)])
alpha = 0.5

def computeError(params):
    a = params[0]
    b = params[1]
    c = params[2]
    d = params[3]
    e = params[4]
    transfo = np.identity(3, dtype = float)
    transfo[0][0] = a
    transfo[0][2] = b
    transfo[1][1] = c
    transfo[2][0] = d
    transfo[2][2] = e
    
    error = 0.0;

    Nsample = 256.0
    
    i = np.arange(0, Nsample)
    j = np.arange(0, Nsample)
    i, j = np.meshgrid(i, j)

    U1 = (i+0.5)/Nsample;
    U2 = (j+0.5)/Nsample;

#     a2 = alpha * alpha;
#     phi = np.pi * 2 * U2;
#     cosTheta = np.sqrt(np.maximum(0, (1 - U1)) / (1 + (a2 - 1) * U1))
#     sinTheta = np.sqrt(np.maximum(0, 1 - cosTheta * cosTheta));

#     Wh = np.array([sinTheta * np.cos(phi), sinTheta * np.sin(phi), cosTheta])
#     Wi = reflectVec3(Wo, Wh);

#     pdf = D_GGX(Wh, alpha) * Wh[2] / (4 * dotVec3(Wh, Wi))


    theta = np.arccos(1-U1);
    phi = U2 * 2 * np.pi
    pdf = 1 / (2 * np.pi)
    Wi = np.array([np.sin(theta) * np.cos(phi), np.sin(theta) * np.sin(phi), np.cos(theta)])

    brdf = BRDF(Wo, Wi, alpha)
    ltc = D_ltc(Wi, transfo)
    error = np.abs(brdf - ltc) ** 2 / pdf

    return np.sum(error) / (Nsample*Nsample)

# TODO: make it work
initial_guess = [1,0,1,0,1]
result = optimize.minimize(computeError, initial_guess, method="Nelder-Mead")
print(result)


Unfortunately above code doesn't work due to time frame constraint, so that I will used data generated by code from this [github repository](https://github.com/selfshadow/ltc_code) instead. Let's validate the fitting.

In [ ]:
from ltc_lut import lut_size, ltc_matrix, ltc_amplitude

def sampleLut(roughness, costheta):
    uvscale = (lut_size - 1.0) / lut_size
    uvbias = 0.5 / lut_size
    uv = np.array([roughness, costheta]) * uvscale + uvbias
    st = uv * lut_size
    iuv = np.floor(st)
    fuv = st - iuv
    
    a = ltc_matrix[int(iuv[1]), int(iuv[0])]
    b = ltc_matrix[int(iuv[1]), np.minimum(63, int(iuv[0] + 1))]
    c = ltc_matrix[np.minimum(63, int(iuv[1] + 1)), int(iuv[0])]
    d = ltc_matrix[np.minimum(63, int(iuv[1] + 1)), np.minimum(63, int(iuv[0]) + 1)]
    lerp = lambda t, a, b: (1.0 - t) * a + t * b
    M = lerp(fuv[1], lerp(fuv[0], a, b), lerp(fuv[0], c, d))
    M = np.transpose(M)
    return M, np.linalg.inv(M)


thetaO = np.pi * (70.0/180)
Wo = np.array([np.sin(thetaO), 0, np.cos(thetaO)])
roughness = 0.4
transfo, inv_transfo = sampleLut(roughness, np.cos(thetaO))


def plot_ltc_error():
    theta, phi = meshgrid_spherical_coord(256)
    x, y, z = spherical_dir(theta, phi)
    Wi = np.array([x, y, z])
    
    ltc = D_ltc(Wi, transfo)
    brdf = BRDF(Wo, Wi, roughness*roughness)
    error = np.abs(ltc - brdf)
    error /= np.maximum(brdf, 1)
#     ipv.plot_mesh(x*ltc, z*ltc, y*ltc, wireframe=False, color=cm.coolwarm(ltc))
#     ipv.plot_mesh(x*brdf, z*brdf, y*brdf, wireframe=False, color=cm.coolwarm(brdf))
    ipv.plot_mesh(x*error, z*error, y*error, wireframe=False, color=cm.coolwarm(error))
    ipv.xyzlim(-2, 2)
    ipv.show()

ipv.clear()
plot_ltc_error()

## Area light on microfacet shading model with linearly transform cosine

Back to our light transport equation, the equation of microfacet shading model over polygonal light is
$$L(W_o) = \int_{P} L_e F_r(W_o, W_i) \overline{\langle N, W_i \rangle} d{W_i} $$

With a diffusion area light, we can move constants term $L_e$ out of the integral, that leave us a simplified form
$$L(W_o) = L_e \int_{P} F_r(W_o, W_i) \overline{\langle N, W_i \rangle} d{W_i} $$

If we can approximate $F_r(W_o, W_i)$ with LTC, then we can solve integration analytically. Can we?  
We already know that $NDF$ and $G$ term can be approximated with LTC, however the integration of $NDF$ and $G$ is not equal to 1 so that we need to apply a multiplier factor to LTC integration. Regarding the fresnel term, it's been baked into this multiplier factor instead of trying to do fitting directly.

$$
\begin{align}
factor &= \int_\Omega Fr(W_o, W_i) \overline{\langle N, W_i \rangle} d{W_i}                                                                                   \\
       &= \int_\Omega F(W_h) {D(W_h)} G_{vis}(W_o, W_i) \overline{\langle N, W_i \rangle} d{W_i}                                                              \\
       &= \int_\Omega (F0 + (1 - F0) (1 - {\overline{\langle W_h, W_i \rangle})^5}) {D(W_h)} G_{vis}(W_o, W_i) \overline{\langle N, W_i \rangle} d{W_i}        \\
       &= F0 \int_\Omega {D(W_h)} G_{vis}(W_o, W_i) \overline{\langle N, W_i \rangle} d{W_i} + 
          (1 - F0) \int_\Omega (1 - (1 - \overline{\langle W_h, W_i \rangle})^5) {D(W_h)} G_{vis}(W_o, W_i) \overline{\langle N, W_i \rangle} d{W_i}        \\
\end{align}
$$


In [ ]:
## Area light with correct shadow

$$L(W_o) = L_e \int_{P} F_r(W_o, W_i) V(W_i) d{W_i} = L_e \frac {\int_{P} F_r(W_o, W_i) V(W_i) d{W_i}} {\int_{P} F_r(W_o, W_i) d{W_i}} {\int_{P} F_r(W_o, W_i) d{W_i}} $$

Since ${\int_{P} F_r(W_o, W_i) d{W_i}}$ can be solved analytically, then the occlusion term is unknow.  
It's obvious that occlusion term  has value between 0 and 1. Unfortunately it's unlikely to has analytic solution, current game either ignore it or compute it in 1 sample per pixel and run a denoiser on top of the 1spp result.

## Conclusion

LTC can calculate polygonal light analytically. However it's application is not limited by the polygonal light, for any shape of light source, if it is integration has analytic solution on original function, it can be approximated by LTC framework. Beyond the variation on type of light source, original solution is not limited by clamp cosine, it can be any function.

Here are a screenshot to compare the result with offline renderer:  
![ltc_comparison](screenshots/ltc_comparison.png)

In [ ]:
## Future work

* Light culling
* Layered material

In [ ]:
## References

* [RADIATIVE VIEW FACTORS](http://webserver.dmt.upm.es/~isidoro/tc3/Radiation%20View%20factors.pdf)
* [sphere ambient occlusion](http://www.iquilezles.org/www/articles/sphereao/sphereao.htm)
* [Real-Time Polygonal-Light Shading with Linearly Transformed Cosines](https://eheitzresearch.wordpress.com/415-2/)
* [Approximate Fresnel term separately](http://blog.selfshadow.com/publications/s2016-advances/s2016_ltc_fresnel.pdf)
* [Real-Time Area Lighting: a Journey From Research to Production](http://blog.selfshadow.com/publications/s2016-advances/)
* [Real-Time Line- and Disk-Light Shading with Linearly Transformed Cosines](https://labs.unity.com/article/real-time-line-and-disk-light-shading-linearly-transformed-cosines)
* [Improving radiosity solutions through the use of analytically determined form-factors](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.466.963&rep=rep1&type=pdf)
* [Deriving the analytical formula for a diffuseresponse from a polygonal light source](http://miciwan.com/misc/diffuse_area_der.pdf)
* [Geometric Derivation of the Irradiance of Polygonal Lights](https://t.co/uZxgHJ3Upz?amp=1)